In [1]:
import os
os.chdir('../')

In [2]:
import pickle
import itertools

from datetime import datetime
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.metrics import roc_auc_score, precision_score, recall_score
from sklearn.model_selection import train_test_split, KFold
from xgboost.sklearn import XGBClassifier

In [3]:
data = pd.read_csv('data/cac40_v3.csv').drop('Unnamed: 0', axis=1)
data

,TICKER,annee,mois,jour,OP,UP,DO,CL,VO,RDMT_J,...,strategique,transaction,actionnaires,coentreprise,construction,collaboration,developpement,l_acquisition,participation,recommandation
0,AC,2007,10,1,50.820,52.450,50.720,52.440,939518,0.045576,...,0,0,0,0,0,0,0,0,0,0
1,AC,2007,10,10,57.600,57.900,56.480,56.740,1167710,0.005640,...,0,0,0,0,0,0,0,0,0,0
2,AC,2007,10,11,56.570,57.900,56.570,57.060,910369,-0.005258,...,0,0,0,0,0,0,0,0,0,0
3,AC,2007,10,12,57.100,57.100,55.500,56.760,996349,-0.011628,...,0,0,0,0,0,0,0,0,0,0
4,AC,2007,10,15,56.760,57.290,55.770,56.100,573647,-0.003209,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
106537,VIV,2017,9,4,20.065,20.235,20.030,20.100,2250835,0.002736,...,0,0,0,0,0,0,0,0,0,0
106538,VIV,2017,9,5,20.095,20.290,19.985,20.155,3095238,-0.001985,...,0,0,0,0,0,0,0,0,0,0
106539,VIV,2017,9,6,20.080,20.210,20.015,20.115,2181925,-0.006463,...,0,0,0,0,0,0,0,0,0,0
106540,VIV,2017,9,7,20.185,20.185,19.920,19.985,4038684,-0.002252,...,0,0,0,0,0,0,0,0,0,0


# Filtration des données
***

In [4]:
variables_num_dep = ["RDMT_J", "RDMT_S", "RDMT_M"]
variables_num_explic = ["OP", "UP", "DO", "CL", "VO"] + [f"{var}_{x}" for x in ["J", "S", "M"] for var in ["HISTO", "VOL", "UP", "DO"]]
numeric_variables = variables_num_dep + variables_num_explic
non_numeric_variables = data.columns[~data.columns.isin(numeric_variables)]
words = non_numeric_variables[4:]
descriptive_variables = non_numeric_variables[:4]

### Question 3)

Q : Montrer que, _quelque soit le ticker_, la liste des mots apparaissant sur 400 lignes et dont le rendement mensuel est en moyenne supérieur à 1% est telle que la table dans le sujet.  

In [5]:
# On veut l'effet du mot, indépendamment de ticker
freq_word_ret = {}
for word in words:
    if data[word].sum() > 400:
        mean_return = np.mean(data["RDMT_M"].loc[data[word] == 1])
        num_app = data[word].sum()
        if mean_return > 0.01:
            freq_word_ret[word] = (num_app, mean_return)

In [6]:
df = pd.DataFrame(freq_word_ret).transpose().rename(axis='columns', mapper={0:"Nombre d'apparitions", 1:"Rendement mensuel moyen"})
df["Nombre d'apparitions"] = df["Nombre d'apparitions"].astype('int64')
df.index.name = "Mot"
df

,Nombre d'apparitions,Rendement mensuel moyen
Mot,,
part,716,0.011925
plus,418,0.010119
pour,1966,0.013645
euros,446,0.010361
group,493,0.012940
passe,527,0.013763
titre,467,0.012178
groupe,467,0.014646
nouvel,494,0.011500


<b>Filtrer la base sur l'apparition de ces mots</b>

In [7]:
keepers = list(freq_word_ret.keys())
data["KEEP_ROW"] = data[keepers].sum(axis=1)
data = data.loc[~(data["KEEP_ROW"] == 0)].drop("KEEP_ROW", axis=1).reset_index(drop=True)

In [8]:
data.shape

(7129, 280)

### Question 4)

Q : Retrait des variables trop corrélées (positivement ou négativement).

In [9]:
CORRELATION_THRESHOLD = 0.75
correlation_matrix = data.loc[:,~data.columns.isin(["TICKER", "annee", "mois", "jour"])].corr().abs()
upper_triangle = correlation_matrix.where(np.triu(np.ones(correlation_matrix.shape), k=1).astype(np.bool))
high_corr_vars = [col for col in upper_triangle.columns if any(upper_triangle[col] > CORRELATION_THRESHOLD)]

In [10]:
data.drop(high_corr_vars, axis=1, inplace=True)
data

,TICKER,annee,mois,jour,OP,VO,RDMT_J,RDMT_S,RDMT_M,HISTO_J,...,strategique,transaction,actionnaires,coentreprise,construction,collaboration,developpement,l_acquisition,participation,recommandation
0,AC,2010,10,11,27.370,583747,0.020663,0.065797,0.123255,0.009700,...,0,0,0,0,0,0,0,0,0,0
1,AC,2010,10,28,29.220,752703,0.011500,0.053896,0.117920,-0.003251,...,0,0,0,0,0,0,0,0,0,0
2,AC,2010,10,6,26.950,1747197,-0.022407,0.042645,0.132093,0.030540,...,0,0,0,0,0,0,0,0,0,0
3,AC,2010,11,26,33.415,871648,-0.022133,-0.012626,0.001485,0.001339,...,0,0,0,0,0,0,0,0,0,0
4,AC,2010,11,3,29.745,880633,0.029165,0.040898,0.126383,0.004715,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7124,VIV,2017,9,20,21.075,2787945,-0.004282,-0.011180,-0.025690,-0.007085,...,0,0,0,0,0,0,0,0,0,0
7125,VIV,2017,9,25,20.710,2082640,-0.001445,0.037581,-0.002891,0.000482,...,0,0,0,0,0,0,0,0,0,0
7126,VIV,2017,9,26,20.765,1622952,0.002895,0.032569,-0.008203,-0.001445,...,0,0,0,0,0,0,0,0,0,0
7127,VIV,2017,9,29,21.060,6358735,0.005369,-0.012138,0.014006,0.029066,...,0,0,0,0,0,0,0,0,0,0


### Resample data into volume bars

In [11]:
total_daily_volume = data.groupby(['annee', 'mois', 'jour']).sum()["VO"]
cumul_daily_volume = total_daily_volume.cumsum()

data["TOTAL_VOLUME_DAILY"] = data.apply(lambda x: total_daily_volume[x["annee"], x["mois"], x["jour"]], axis=1)
data["CUMUL_VOLUME_DAILY"] = data.apply(lambda x: cumul_daily_volume[x["annee"], x["mois"], x["jour"]], axis=1)

In [12]:
data.loc[(data["annee"]==2008) & (data["mois"]==1) & (data["jour"]==21)].head()

,TICKER,annee,mois,jour,OP,VO,RDMT_J,RDMT_S,RDMT_M,HISTO_J,...,actionnaires,coentreprise,construction,collaboration,developpement,l_acquisition,participation,recommandation,TOTAL_VOLUME_DAILY,CUMUL_VOLUME_DAILY
2273,EN,2008,1,21,49.49,3697773,0.032205,0.069983,0.017754,-0.036787,...,0,0,0,0,0,0,0,0,3697773,6589168


In [18]:
cumul_daily_volume

annee  mois  jour
2008   1     16          2891395
             21          6589168
             24          9969483
             28         13404139
             29         15936256
                        ...     
2017   10    23      20948693204
             24      20956445762
             25      20970281973
             26      20988213675
             27      21002071104
Name: VO, Length: 2141, dtype: int64

In [17]:
total_daily_volume

annee  mois  jour
2008   1     16       2891395
             21       3697773
             24       3380315
             28       3434656
             29       2532117
                       ...   
2017   10    23      14180600
             24       7752558
             25      13836211
             26      17931702
             27      13857429
Name: VO, Length: 2141, dtype: int64

In [20]:
total_data_volume = cumul_daily_volume.iloc[-1]
f"{total_data_volume:,.4f}"

'21,002,071,104.0000'

In [21]:
VOL_THRESHOLD = 1000000000

traded_volume = 0
days_to_keep = []
for idx in cumul_daily_volume.index:
    traded_volume += cumul_daily_volume[idx]
    if traded_volume > VOL_THRESHOLD:
        days_to_keep.append(idx)
        traded_volume = 0

In [22]:
print(len(days_to_keep)), print(len(cumul_daily_volume))

1997
2141


(None, None)

In [23]:
keeper_idx = []
for idx in data.index:
    date = tuple(data.loc[idx, ["annee", "mois", "jour"]])
    if date in days_to_keep:
        keeper_idx.append(idx)

In [30]:
sample = data.iloc[keeper_idx,:].copy()
sample.shape

(6924, 249)

## Train model

In [31]:
LABEL = "RDMT_M"
LABEL_THRESHOLD = 0.02

sample.drop(["TICKER"], inplace=True, axis='columns')

X = sample.loc[:,~sample.columns.isin(variables_num_dep)].copy()
y = sample[LABEL].copy()
y_label = pd.DataFrame({'y':[1 if v > LABEL_THRESHOLD else -1 for v in y]})
y_label.name = "LABEL"

In [32]:
X.sort_values(by=['annee', 'mois', 'jour'])

,annee,mois,jour,OP,VO,HISTO_J,HISTO_S,HISTO_M,VOL_J,VOL_S,...,actionnaires,coentreprise,construction,collaboration,developpement,l_acquisition,participation,recommandation,TOTAL_VOLUME_DAILY,CUMUL_VOLUME_DAILY
981,2008,3,12,53.40,5754537,0.039186,0.041586,0.018449,0.948054,1.483838,...,0,0,0,0,0,0,0,1,5754537,82602101
2284,2008,4,21,47.59,1253412,-0.013064,0.063337,0.128403,-0.318421,-0.250308,...,0,0,0,0,0,0,0,0,1253412,113687569
2919,2008,5,21,8.93,1460553,-0.027068,-0.037918,0.043132,0.352165,0.923013,...,0,0,0,0,0,0,0,0,1460553,133160354
4278,2008,6,9,195.45,1101912,0.015800,0.048282,0.073258,-0.492445,0.057369,...,0,0,0,0,0,0,0,0,1699147,150835794
6341,2008,6,9,44.45,597235,-0.002667,-0.000445,0.033149,-0.293931,0.745107,...,0,0,0,0,0,0,0,0,1699147,150835794
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2868,2017,10,27,46.59,8383356,0.014922,0.028503,0.047883,0.527491,0.193948,...,0,0,0,0,0,0,0,0,13857429,21002071104
3791,2017,10,27,47.80,909449,-0.007025,-0.012954,-0.006823,1.348759,0.874283,...,0,0,0,0,0,0,0,0,13857429,21002071104
4246,2017,10,27,126.30,493303,0.006354,0.046243,0.020540,-0.115704,-0.289339,...,0,0,0,0,0,0,0,0,13857429,21002071104
5542,2017,10,27,90.25,1436101,0.015361,0.056521,0.072167,0.216967,0.465092,...,0,0,0,0,0,0,0,0,13857429,21002071104


In [33]:
X_train, X_test, y_train, y_test = train_test_split(X, y_label, test_size=0.2)

In [35]:
print(X_train.shape)
print(X_test.shape)

(5539, 245)
(1385, 245)


In [36]:
import itertools

hyper_learning_rate = np.logspace(-4,-2, 4)
hyper_max_depth = [8, 9, 10, 11, 12]
hyper_n_estimators = [100, 200, 300]
hyper_subsample = [0.7, 0.8, 0.9, 1]
hyper_colsample_bytree = [0.7, 0.8, 0.9, 1]
hyper_colsample_bylevel = [0.7, 0.8, 0.9, 1]
hyper_gamma = [1, 5, 10, 20]

hyper_parameter_list = [hyper_learning_rate, hyper_max_depth, 
                        hyper_n_estimators, hyper_subsample, 
                        hyper_colsample_bytree, hyper_colsample_bylevel,
                        hyper_gamma]

hyper_parameter_grid = list(itertools.product(*hyper_parameter_list))

len(hyper_parameter_grid)

15360

In [37]:
from helper_functions import tune_XGB_hyper_params

In [ ]:
#results = tune_XGB_hyper_params(hyper_parameter_grid, X_train, y_train, 4, 100)

In [40]:
#with open("results_tuning_results_volume_bars_2.pickle", "wb") as file:
#    pickle.dump(results, file)

with open("results_tuning_results_volume_bars_2.pickle", "rb") as file:
    results = pickle.load(file)

In [41]:
idx = np.argmax([x['metriques']['auc'] for x in results])
max_result = results[idx]
max_result

{'algorithm': 'XGBClassifier',
 'parametres': {'learning_rate': 0.01,
  'max_depth': 12,
  'n_estimators': 200,
  'subsample': 0.7,
  'colsample_bytree': 1,
  'colsample_bylevel': 0.7,
  'gamma': 1},
 'metriques': {'auc': 0.9667, 'precision': 0.9593, 'recall': 0.9673}}

In [42]:
from xgboost.sklearn import XGBClassifier
from sklearn.metrics import roc_auc_score

In [43]:
max_params = max_result['parametres']
xgb = XGBClassifier(booster='gbtree', objective='binary:logistic',
                    learning_rate=max_params['learning_rate'],
                    max_depth=max_params['max_depth'],
                    n_estimators=max_params['n_estimators'],
                    subsample=max_params['subsample'],
                    colsample_bytree=max_params['colsample_bytree'],
                    colsample_bylevel=max_params['colsample_bylevel'])


In [44]:
model = xgb.fit(X_train, y_train.y)
y_hat = model.predict(X_test)
auc = roc_auc_score(y_test.y, y_hat)
auc

0.7029838723141393